In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [8]:
try:
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    print(f"Language Model initialized: {llm.model}")
except Exception as e:
    print(f"Error initilaizing language model : {e}")

Language Model initialized: models/gemini-2.5-flash


In [10]:
def booking_handler(request: str) -> str :
    print("\n -- DELEGATING TO BOOKING HANDLER    --")
    return f"""
                Booking Handler processed request:'{request}' Result: simulated booking action
            """

In [11]:
def info_handler(request: str) -> str:
    print(f"\n -- DELEGATING TO INFO HANDLER  -- \n")
    return f"""
            Info handler processed the request : '{request}' Result: Simulated information retrieval
    """

In [12]:
def unclear_handler(request: str) -> str:
    print("\n -- HANDLING UNCLER REQUEST --  \n")
    return f"""
            Coordinator could not delegate request : '{request}'. Please clarify
    """

In [15]:
coordinator_router_prompt = ChatPromptTemplate.from_messages([
                            ( "system", 
                                """
                                Analyze the user's request and determine which specialist handler should process it.   
                                - If the request is related to booking flights or hotels,      
                                output 'booker'.    - For all other general information questions, output 'info'.  
                                - If the request is unclear or doesn't fit either category,       output 'unclear'.     
                                ONLY output one word: 'booker', 'info', or 'unclear'.
                                """
                            ),
                            (
                                "user",
                                "{request}"
                                                                
                            )
                            
                                
                            ])

In [17]:
if llm:
    coordinator_router_chain = coordinator_router_prompt | llm | StrOutputParser()

In [25]:
branches = {
             "booker": RunnablePassthrough.assign(output=lambda x: booking_handler(x['request']['request'])),
             "info":    RunnablePassthrough.assign(output=lambda x: info_handler(x['request']['request'])),
             "unclear": RunnablePassthrough.assign(output=lambda x: unclear_handler(x['request']['request']))
           }


                                    

In [27]:
# Create the RunnableBranch. It takes the output of the router chain # and routes the original input ('request') to the corresponding handler. 

delegation_branch = RunnableBranch( (lambda x: x['decision'].strip() == 'booker', branches["booker"]), 
                                    (lambda x: x['decision'].strip() == 'info', branches["info"]), 
                                     branches["unclear"]
                                  )

In [29]:
coordinator_agent = {
                        "decision": coordinator_router_chain,
                        "request": RunnablePassthrough()

                    } | delegation_branch | (lambda x: x['output']) 

In [34]:
def main():
    if not llm:
        print("\n Skipping execution due to LLM inititalization failure")
        return
    print("\n --- Running with a booking request --")
    request_a = "Book me a flight to London"
    result_a = coordinator_agent.invoke({"request": request_a})
    print(f"Final Result A: {result_a}")

    print("\n --- Running with a info request --")
    request_b = "What is capital of Italy"
    result_b = coordinator_agent.invoke({"request": request_b})
    print(f"Final Result B: {result_b}")

    print("\n --- Running with a unclear request --")
    request_c = "Tell me about quantum physics"
    result_c = coordinator_agent.invoke({"request": request_c})
    print(f"Final Result C: {result_c}")

    
    

In [35]:
main()


 --- Running with a booking request --

 -- DELEGATING TO BOOKING HANDLER    --
Final Result A: 
                Booking Handler processed request:'Book me a flight to London' Result: simulated booking action
            

 --- Running with a info request --

 -- DELEGATING TO INFO HANDLER  -- 

Final Result B: 
            Info handler processed the request : 'What is capital of Italy' Result: Simulated information retrieval
    

 --- Running with a unclear request --

 -- DELEGATING TO INFO HANDLER  -- 

Final Result C: 
            Info handler processed the request : 'Tell me about quantum physics' Result: Simulated information retrieval
    
